In [1]:
import sys
import os
from datetime import datetime
import pyarrow.parquet as pq
import time
from utils import config
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sankey import Sankey
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

# Import tableone
try:
    from tableone import TableOne, load_dataset
except (ModuleNotFoundError, ImportError):
    # install on Colab
    !pip install tableone
    from tableone import TableOne, load_dataset

Loaded configuration from config.json


## please change site_name, tables_path, output_path, and file_type 

In [ ]:
# Access configuration parameters
site_name = config['site_name']
tables_path = config['tables_path']
file_type = config['file_type']
output_path = os.path.join("..", "output", "final")

# Make sure the directory exists; if not, create it
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Print the configuration parameters
print(f"Site Name: {site_name}")
print(f"Tables Path: {tables_path}")
print(f"Output path: {output_path}")
print(f"File Type: {file_type}")
                   
## Confirm that these are the correct paths
adt_filepath = f"{tables_path}/clif_adt.{file_type}"
hospitalization_filepath = f"{tables_path}/clif_hospitalization.{file_type}"
patient_filepath = f"{tables_path}/clif_patient.{file_type}"

In [ ]:
def read_data(filepath, filetype):
    start_time = time.time()  # Record the start time
    file_name = os.path.basename(filepath) 
    if filetype == 'csv':
        df = pd.read_csv(filepath)
    elif filetype == 'parquet':
        table = pq.read_table(filepath)
        df = table.to_pandas()
    else:
        raise ValueError("Unsupported file type. Please provide either 'csv' or 'parquet'.")
    
    end_time = time.time()  # Record the end time
    load_time = end_time - start_time  # Calculate the loading time
    
    # Calculate the size of the loaded dataset in MB
    dataset_size_mb = df.memory_usage(deep=True).sum() / (1024 * 1024)
    print(f"File name: {file_name}")
    print(f"Time taken to load the dataset: {load_time:.2f} seconds")
    print(f"Size of the loaded dataset: {dataset_size_mb:.2f} MB\n")
    
    return df

adt = read_data(adt_filepath, file_type)
hosp = read_data(hospitalization_filepath, file_type)
patient = read_data(patient_filepath,file_type)

In [ ]:
print("number of unique hospitalization_id:",hosp["hospitalization_id"].nunique())

In [ ]:
missing_admission_dttm = hosp['admission_dttm'].isnull().sum()
print("Missing admission_dttm:", missing_admission_dttm)

In [ ]:
hosp2 = hosp[["patient_id","hospitalization_id","admission_dttm","discharge_dttm","age_at_admission"]].copy()
hosp2['admission_dttm'] = pd.to_datetime(hosp2['admission_dttm'])
hosp2['discharge_dttm'] = pd.to_datetime(hosp2['discharge_dttm'])
hosp2 = hosp2[(hosp2['admission_dttm'].dt.year >= 2020) & 
                   (hosp2['admission_dttm'].dt.year <= 2021) & 
                   (hosp2['age_at_admission'] >=18)&
                    (hosp2['age_at_admission'] <=119)]

df = pd.merge(hosp2[["patient_id","hospitalization_id"]],
                  adt[["hospitalization_id","in_dttm","out_dttm","location_category"]],
             on="hospitalization_id",how="left")

print("number of unique hospitalization_id:",df["hospitalization_id"].nunique())
print("number of rows in df:",len(df))
df.head()

In [ ]:
df["location_category"] = df["location_category"].str.lower().str.strip()
# Mapping dictionary to update the location_category
location_mapping = {
    'rehab': 'Other',
    'l&d': 'Ward',
    'psych': 'Other',
    'radiology': 'Other',
    'dialysis': 'Other',
    'hospice': 'Other',
    'stepdown':'ICU',
    "icu": "ICU",
    "er": "ER",
    "ward":"Ward",
    "procedural":"Procedural",
    "other":"Other"
}

# Apply the mapping to the 'location_category' column
df['location_category'] = df['location_category'].replace(location_mapping)
df["location_category"].value_counts()

In [ ]:
# Step 1: Sort by hospitalization_id and in_dttm
df = df.sort_values(by=['hospitalization_id', 'in_dttm', 'out_dttm'])
df = df[df['location_category']!= "Other"] #removing "Other from the study"

# Step 2: Group by hospitalization_id and apply the removal logic
def remove_procedural_pattern(group):
    # Loop through the group to find the ICU --> Procedural --> ICU pattern
    to_remove = []
    
    for i in range(1, len(group) - 1):  # Start from second row and end before the last row
        # Check for ICU -> Procedural -> ICU pattern
        if (group.iloc[i-1]['location_category'] == 'ICU' and
            group.iloc[i]['location_category'] == 'Procedural' and
            group.iloc[i+1]['location_category'] == 'ICU'):
            # Mark the 'Procedural' row for removal
            to_remove.append(group.index[i])
    
    # Return the group without the rows to be removed
    return group.drop(to_remove)

# Apply the function to each group of hospitalization_id
df = df.sort_values(by=['hospitalization_id', 'in_dttm', 'out_dttm'])
#stitch together ICUs if ICU --> procedure --> ICU
df['shifted_location'] = df['location_category'].shift(1)
df['new_location'] = (df['location_category'] != df['shifted_location']).cumsum()
df = df.groupby(['hospitalization_id','new_location']).agg(
    in_dttm=pd.NamedAgg(column='in_dttm', aggfunc='min'),
    out_dttm=pd.NamedAgg(column='out_dttm', aggfunc='max'),
    location_category=pd.NamedAgg(column='location_category', aggfunc='first'), 
    patient_id=pd.NamedAgg(column='patient_id', aggfunc='first')
).reset_index()
df = df.groupby('hospitalization_id').apply(remove_procedural_pattern).reset_index(drop=True)

print("Number of unique hospitalization_id:", df["hospitalization_id"].nunique())
print("Number of rows in df:",len(df))

In [ ]:
def remove_short_stays(df):
    # Ensure that the in_dttm and out_dttm columns are in datetime format
    df['in_dttm'] = pd.to_datetime(df['in_dttm'])
    df['out_dttm'] = pd.to_datetime(df['out_dttm'])
    
    # Calculate the time difference between out_dttm and in_dttm
    df['stay_duration'] = df['out_dttm'] - df['in_dttm']
    
    # Filter out rows where the stay duration is less than 1 hour
    df_filtered = df[df['stay_duration'] >= pd.Timedelta(hours=1)]
    
    # Drop the stay_duration column as it is no longer needed
    df_filtered = df_filtered.drop(columns=['stay_duration'])
    
    return df_filtered

# Sort the DataFrame as needed
df = df.sort_values(by=['hospitalization_id', 'in_dttm', 'out_dttm'])

# Apply the function to the entire DataFrame, not with .apply()
df = remove_short_stays(df)

# Perform further transformations
df['shifted_location'] = df['location_category'].shift(1)
df['new_location'] = (df['location_category'] != df['shifted_location']).cumsum()

# Group by hospitalization_id and new_location, and aggregate
df = df.groupby(['hospitalization_id', 'new_location']).agg(
    in_dttm=pd.NamedAgg(column='in_dttm', aggfunc='min'),
    out_dttm=pd.NamedAgg(column='out_dttm', aggfunc='max'),
    location_category=pd.NamedAgg(column='location_category', aggfunc='first'),
    patient_id=pd.NamedAgg(column='patient_id', aggfunc='first')
).reset_index()

# Print summary information
print("Number of unique hospitalization_id:", df["hospitalization_id"].nunique())
print("Number of rows in df:", len(df))

In [ ]:
df['in_dttm'] = pd.to_datetime(df['in_dttm'])
df['out_dttm'] = pd.to_datetime(df['out_dttm'])
df = df.sort_values(by=['hospitalization_id', 'in_dttm', 'out_dttm'])

def mark_procedural_before_second_icu(df):
    # Initialize the 'procedural' column with False
    df['procedural'] = False

    # Function to process each group of 'patient_id' and 'hospitalization_id'
    def process_group(group):
        # Find all the rows where the location is ICU
        icu_indices = group[group['location_category'] == 'ICU'].index

        # Check if at least two ICU admissions exist in the group
        if len(icu_indices) > 1:
            # Get the index of the second ICU admission
            second_icu_index = icu_indices[1]

            # Check the row just before the second ICU admission
            if second_icu_index > group.index[0]:
                row_before_second_icu = group.iloc[second_icu_index - group.index[0] - 1]

                # If the row before the second ICU is "Procedural", mark the entire group as procedural
                if row_before_second_icu['location_category'] == 'Procedural':
                    df.loc[group.index, 'procedural'] = True  # Mark the entire group as procedural

    # Apply the function to each group of 'patient_id' and 'hospitalization_id'
    df.groupby(['patient_id', 'hospitalization_id']).apply(process_group)

    return df

df2 = df.sort_values(by=['hospitalization_id', 'in_dttm', 'out_dttm'])
#making sure the dataset is aggregated correctly
df2['shifted_location'] = df2['location_category'].shift(1)
df2['new_location'] = (df2['location_category'] != df2['shifted_location']).cumsum()
df2 = df2.groupby(['hospitalization_id','new_location']).agg(
    in_dttm=pd.NamedAgg(column='in_dttm', aggfunc='min'),
    out_dttm=pd.NamedAgg(column='out_dttm', aggfunc='max'),
    location_category=pd.NamedAgg(column='location_category', aggfunc='first'), 
    patient_id=pd.NamedAgg(column='patient_id', aggfunc='first')
).reset_index()
df2 = mark_procedural_before_second_icu(df2)

# Output the updated DataFrame
print("Number of unique hospitalization_id:", df2["hospitalization_id"].nunique())
print("Number of rows in df:", len(df2))
print("Number of hospitalizations marked as procedural:", df2[df2["procedural"]==True]["hospitalization_id"].nunique())
df2[['patient_id', 'hospitalization_id', 'location_category', 'procedural']].head()

In [ ]:
df2['in_dttm'] = pd.to_datetime(df2['in_dttm'])
df2['out_dttm'] = pd.to_datetime(df2['out_dttm'])
df2 = df2.sort_values(by=['hospitalization_id', 'in_dttm', 'out_dttm'])

# Function to filter visit_occurrence_ids based on ICU location
def filter_icu_visits(df):
    # Find all rows where location is ICU
    icu_visits = df[df['location_category'] == "ICU"]

    # If there are no ICU visits, return an empty DataFrame
    if icu_visits.empty:
        return pd.DataFrame(columns=df.columns)
    
    # Get the unique visit_occurrence_ids that include ICU
    icu_visit_ids = icu_visits['hospitalization_id'].unique()
    
    # Filter to include only rows with these visit_occurrence_ids (all ICU visits)
    df_filtered = df[df['hospitalization_id'].isin(icu_visit_ids)]
    
    return df_filtered

# Apply the function to each visit occurence to keep all ICU visits
icu_df = df2.groupby('hospitalization_id').apply(filter_icu_visits).reset_index(drop=True)
#icu_df = icu_df[icu_df['location_category'] != "Procedural"]
icu_df = icu_df.sort_values(by=['hospitalization_id', 'in_dttm','out_dttm'])

print("Number of unique hospitalization_id:", icu_df["hospitalization_id"].nunique())
print("Number of rows in df:",len(icu_df))

In [ ]:
def drop_deaths_during_icu(df, patient_df):
    # Merge death_dttm from patient data into icu_df
    df = pd.merge(df, patient_df[["patient_id", "death_dttm"]], on="patient_id", how="left")

    # Ensure the columns 'in_dttm' and 'out_dttm' are in datetime format
    df['death_dttm'] = pd.to_datetime(df['death_dttm'], errors='coerce')
    df['in_dttm'] = pd.to_datetime(df['in_dttm'], errors='coerce')
    df['out_dttm'] = pd.to_datetime(df['out_dttm'], errors='coerce')

    # Sort the DataFrame by hospitalization_id and in_dttm for correct processing
    df = df.sort_values(by=['hospitalization_id', 'in_dttm'])

    # Identify first ICU admission per hospitalization
    icu_first_admission = df[df['location_category'] == 'ICU'].groupby('hospitalization_id').first().reset_index()

    # Identify hospitalization_id where the patient died during ICU admission
    death_during_icu = icu_first_admission[
        (icu_first_admission['death_dttm'].notnull()) &
        (icu_first_admission['death_dttm'] >= icu_first_admission['in_dttm']) &
        (icu_first_admission['death_dttm'] <= icu_first_admission['out_dttm'])
    ]['hospitalization_id']

    # Drop rows where the patient died during ICU admission
    dropped_df = df[df['hospitalization_id'].isin(death_during_icu)]
    df_cleaned = df[~df['hospitalization_id'].isin(death_during_icu)]

    return df_cleaned, dropped_df

icu_df = icu_df.sort_values(by=['hospitalization_id', 'in_dttm'])
icu_df2, dropped_deaths_df = drop_deaths_during_icu(icu_df, patient)

print("Number of unique hospitalization_id with death during ICU :",dropped_deaths_df["hospitalization_id"].nunique())
print("Number of unique hospitalization_id after removing death during ICU:", icu_df2["hospitalization_id"].nunique())
print("Number of rows in df:",len(icu_df2))

In [ ]:
def drop_direct_icu_discharges(df):
    # Initialize an empty list to hold custom_visit_occurrence2 values to be dropped
    visits_to_drop = []

    # Function to process each group of 'person_id' and 'custom_visit_occurrence2'
    def process_group(group):
        # Find all the rows where the location is ICU
        icu_indices = group[group['location_category'] == 'ICU'].index

        # Check if any ICU admissions exist in the group
        if not icu_indices.empty:
            # Get the index of the first ICU admission
            first_icu_index = icu_indices[0]

            # Check if this first ICU admission is also the last row in the visit
            if first_icu_index == group.index[-1]:
                # This means the patient was discharged or died directly from the ICU
                visits_to_drop.append(group.iloc[0]['hospitalization_id'])  # Mark the visit to be dropped

    # Apply the function to each group of 'person_id' and 'custom_visit_occurrence2'
    df.groupby(['patient_id', 'hospitalization_id']).apply(process_group)

    # Create a new DataFrame with the rows that need to be dropped
    dropped_df = df[df['hospitalization_id'].isin(visits_to_drop)]

    # Drop the visits from the original DataFrame
    df_cleaned = df[~df['hospitalization_id'].isin(visits_to_drop)]

    return df_cleaned, dropped_df

icu_df2 = icu_df2.sort_values(by=['hospitalization_id', 'in_dttm','out_dttm'])
icu_df3, dropped_df_icu_discharge = drop_direct_icu_discharges(icu_df2)

# Output the counts and dropped DataFrame
print("Number of unique hospitalization_id after filtering:", icu_df3["hospitalization_id"].nunique())
print("Number of rows in df:",len(icu_df3))
print("Number of unique patient_id dropped:", dropped_df_icu_discharge["patient_id"].nunique())
print("Number of unique hospitalization_id dropped:", dropped_df_icu_discharge["hospitalization_id"].nunique())

In [ ]:
#redundant but making sure that the location_category is grouped together 
icu_df4 = icu_df3.sort_values(by=['hospitalization_id', 'in_dttm','out_dttm'])
icu_df4['shifted_location'] = icu_df4['location_category'].shift(1)
icu_df4['new_location'] = (icu_df4['location_category'] != icu_df4['shifted_location']).cumsum()

icu_df4 = icu_df4.groupby(['hospitalization_id','new_location']).agg(
    earliest_location_start=pd.NamedAgg(column='in_dttm', aggfunc='min'),
    latest_location_end=pd.NamedAgg(column='out_dttm', aggfunc='max'),
    location_category=pd.NamedAgg(column='location_category', aggfunc='first'), 
    patient_id=pd.NamedAgg(column='patient_id', aggfunc='first'),
    procedural = pd.NamedAgg(column='procedural', aggfunc='first')
).reset_index()

print("Number of unique hospitalization_id:", icu_df4["hospitalization_id"].nunique())
icu_df4.head()

In [ ]:
def drop_intermediate_er(df):
    # Initialize an empty list to hold hospitalization_id values to be dropped
    visits_to_drop = []

    # Function to process each group of 'patient_id' and 'hospitalization_id'
    def process_group(group):
        # Check if 'ER' appears after the first row in the group
        if any(group.iloc[1:]['location_category'] == 'ER'):
            # Mark the visit for dropping if 'ER' appears after the first row
            visits_to_drop.append(group.iloc[0]['hospitalization_id'])

    # Apply the function to each group of'hospitalization_id'
    df.groupby(['hospitalization_id']).apply(process_group)

    # Create a new DataFrame with the rows that need to be dropped
    dropped_df = df[df['hospitalization_id'].isin(visits_to_drop)]

    # Drop the visits from the original DataFrame
    df_cleaned = df[~df['hospitalization_id'].isin(visits_to_drop)]

    return df_cleaned, dropped_df

# Sort and apply the drop function
icu_df4 = icu_df4.sort_values(by=['hospitalization_id', 'earliest_location_start', 'latest_location_end'])
icu_df5, dropped_df = drop_intermediate_er(icu_df4)

# Convert to datetime and calculate location hours
icu_df5['earliest_location_start'] = pd.to_datetime(icu_df5['earliest_location_start'])
icu_df5['latest_location_end'] = pd.to_datetime(icu_df5['latest_location_end'])
icu_df5['location_hours'] = (icu_df5['latest_location_end'] - icu_df5['earliest_location_start']).dt.total_seconds() / 3600

# Output the results
print("Number of rows in df:", len(icu_df5))
print("Number of unique hospitalization_id:", icu_df5["hospitalization_id"].nunique())
icu_df5.head()

# creating the readmission flag

In [ ]:
def calculate_icu_readmission_time(df):
    # Initialize a new column 'ICU_readmission_hour' with None
    df['ICU_readmission_hour'] = None

    # Function to process each group
    def process_group(group):
        # Find the rows where location_category is ICU
        icu_rows = group[group['location_category'] == 'ICU']

        # Check if there are at least two ICU events
        if len(icu_rows) > 1:
            # Sort by earliest_start2 to ensure correct order
            icu_rows = icu_rows.sort_values(by='earliest_location_start')

            # Get the first and second ICU event
            first_icu = icu_rows.iloc[0]
            second_icu = icu_rows.iloc[1]

            # Calculate the time difference between first and second ICU events in hours
            time_diff = (second_icu['earliest_location_start'] - first_icu['latest_location_end']).total_seconds() / 3600

            # Update the entire group with the calculated time difference
            df.loc[group.index, 'ICU_readmission_hour'] = time_diff

    # Apply the function to each group of 'person_id' and 'custom_visit_occurrence2'
    df.groupby(['patient_id', 'hospitalization_id']).apply(process_group)

    return df

icu_df6 = calculate_icu_readmission_time(icu_df5)
print("number of rows in df:",len(icu_df6))
print("Number of unique hospitalization_id:", icu_df6["hospitalization_id"].nunique())

In [ ]:
# Identify hospitalization_ids where ICU_readmission_hour is less than 0
#This is in place incase there are errors in the adt, should be a very small number if any 
hosp_ids_to_drop = icu_df6[icu_df6["ICU_readmission_hour"] <=0]["hospitalization_id"].unique()

# Drop rows with those hospitalization_ids
icu_df7 = icu_df6[~icu_df6["hospitalization_id"].isin(hosp_ids_to_drop)]

# Check the result
print("Number of unique hospitalization_id after filtering:", icu_df7["hospitalization_id"].nunique())
print("Number of rows in df after filtering:", len(icu_df7))

In [ ]:
#pd.set_option('display.max_rows',None)

In [ ]:
#icu_df6[icu_df6["ICU_readmission_hour"]>0][:100]

In [ ]:
# Select relevant columns
icu_final = icu_df7[["patient_id", "hospitalization_id", "ICU_readmission_hour","procedural"]]

# Drop duplicates based on 'custom_visit_occurrence2'
icu_final = icu_final.drop_duplicates(subset=["hospitalization_id"])

# Display the first few rows of the result
print("Number of rows in df:",len(icu_final))
print("Number of unique patient_id:", icu_final["patient_id"].nunique())
print("Number of unique hospitalization_id:", icu_final["hospitalization_id"].nunique())
icu_final.head()

# create table 

In [ ]:
total_unique_visits = icu_final["hospitalization_id"].nunique()
unique_visits_with_procedures = icu_final[icu_final["procedural"]==True]["hospitalization_id"].nunique()
unique_visits_without_procedures = icu_final[icu_final["procedural"]==False]["hospitalization_id"].nunique()
all_icu_readmission = icu_final[(icu_final["ICU_readmission_hour"]>0)]["hospitalization_id"].nunique()
all_icu_readmission_6hours = icu_final[(icu_final["ICU_readmission_hour"]<6)]["hospitalization_id"].nunique()
all_icu_readmission_24hours = icu_final[(icu_final["ICU_readmission_hour"]<24)]["hospitalization_id"].nunique()
all_icu_readmission_48hours = icu_final[(icu_final["ICU_readmission_hour"]<48)]["hospitalization_id"].nunique()
all_icu_readmission_72hours = icu_final[(icu_final["ICU_readmission_hour"]<72)]["hospitalization_id"].nunique()
unplanned_icu_readmission = icu_final[(icu_final["procedural"]==False)&(icu_final["ICU_readmission_hour"]>0)]["hospitalization_id"].nunique()
unplanned_icu_readmission_6hours = icu_final[(icu_final["procedural"]==False)&(icu_final["ICU_readmission_hour"]<6)]["hospitalization_id"].nunique()
unplanned_icu_readmission_24hours = icu_final[(icu_final["procedural"]==False)&(icu_final["ICU_readmission_hour"]<24)]["hospitalization_id"].nunique()
unplanned_icu_readmission_48hours = icu_final[(icu_final["procedural"]==False)&(icu_final["ICU_readmission_hour"]<48)]["hospitalization_id"].nunique()
unplanned_icu_readmission_72hours = icu_final[(icu_final["procedural"]==False)&(icu_final["ICU_readmission_hour"]<72)]["hospitalization_id"].nunique()


print("Number of unique hospitalization_id:", total_unique_visits)
print("Planned readmission: Number of unique hospitalization_id with procedures between ICU readmission:", unique_visits_with_procedures)
print("Unplanned readmission: Number of unique hospitalization_id without any procedures between ICU readmission:", unique_visits_without_procedures)

all_icu_read = round((all_icu_readmission / total_unique_visits)*100,2)
all_6hrs = round((all_icu_readmission_6hours/total_unique_visits)*100,2)
all_24hrs = round((all_icu_readmission_24hours/total_unique_visits)*100,2)
all_48hrs = round((all_icu_readmission_48hours/total_unique_visits)*100,2)
all_72hrs = round((all_icu_readmission_72hours/total_unique_visits)*100,2)

unplanned_icu_read = round((unplanned_icu_readmission / unique_visits_without_procedures)*100,2)
unplanned_6hrs = round((unplanned_icu_readmission_6hours/unique_visits_without_procedures)*100,2)
unplanned_24hrs = round((unplanned_icu_readmission_24hours/unique_visits_without_procedures)*100,2)
unplanned_48hrs = round((unplanned_icu_readmission_48hours/unique_visits_without_procedures)*100,2)
unplanned_72hrs = round((unplanned_icu_readmission_72hours/unique_visits_without_procedures)*100,2)

# Print all readmission rate
print(f"All ICU readmission ({all_icu_readmission}/{total_unique_visits}): {all_icu_read}%")
print(f"All ICU readmission <6hr ({all_icu_readmission_6hours}/{total_unique_visits}): {all_6hrs}%")
print(f"All ICU readmission <24hr ({all_icu_readmission_24hours}/{total_unique_visits}): {all_24hrs}%")
print(f"All ICU readmission <48hr ({all_icu_readmission_48hours}/{total_unique_visits}): {all_48hrs}%")
print(f"All ICU readmission <72hr ({all_icu_readmission_72hours}/{total_unique_visits}): {all_72hrs}%")

# Print unplanned readmission rate
print(f"Unplanned ICU readmission ({unplanned_icu_readmission}/{unique_visits_without_procedures}): {unplanned_icu_read}%")
print(f"Unplanned ICU readmission <6hr ({unplanned_icu_readmission_6hours}/{unique_visits_without_procedures}): {unplanned_6hrs}%")
print(f"Unplanned ICU readmission <24hr ({unplanned_icu_readmission_24hours}/{unique_visits_without_procedures}): {unplanned_24hrs}%")
print(f"Unplanned ICU readmission <48hr ({unplanned_icu_readmission_48hours}/{unique_visits_without_procedures}): {unplanned_48hrs}%")
print(f"Unplanned ICU readmission <72hr ({unplanned_icu_readmission_72hours}/{unique_visits_without_procedures}): {unplanned_72hrs}%")

In [ ]:
data = {
    'Metric': [
        'All ICU Readmission', 
        'All ICU Readmission <6hr', 
        'All ICU Readmission <24hr', 
        'All ICU Readmission <48hr', 
        'All ICU Readmission <72hr', 
        'Unplanned ICU Readmission', 
        'Unplanned ICU Readmission <6hr', 
        'Unplanned ICU Readmission <24hr', 
        'Unplanned ICU Readmission <48hr', 
        'Unplanned ICU Readmission <72hr'
    ],
    'Numerator/Denominator': [
        f"{all_icu_readmission}/{total_unique_visits}", 
        f"{all_icu_readmission_6hours}/{total_unique_visits}", 
        f"{all_icu_readmission_24hours}/{total_unique_visits}", 
        f"{all_icu_readmission_48hours}/{total_unique_visits}", 
        f"{all_icu_readmission_72hours}/{total_unique_visits}", 
        f"{unplanned_icu_readmission}/{unique_visits_without_procedures}", 
        f"{unplanned_icu_readmission_6hours}/{unique_visits_without_procedures}", 
        f"{unplanned_icu_readmission_24hours}/{unique_visits_without_procedures}", 
        f"{unplanned_icu_readmission_48hours}/{unique_visits_without_procedures}", 
        f"{unplanned_icu_readmission_72hours}/{unique_visits_without_procedures}"
    ],
    'Percentage (%)': [
        all_icu_read, 
        all_6hrs, 
        all_24hrs, 
        all_48hrs, 
        all_72hrs, 
        unplanned_icu_read, 
        unplanned_6hrs, 
        unplanned_24hrs, 
        unplanned_48hrs, 
        unplanned_72hrs
    ]
}
# Create the DataFrame
final_df = pd.DataFrame(data)
final_df.to_csv(f"{output_path}ICU_readmission_rates_{datetime.today().date()}_{site_name}.csv")
final_df

# Generate Sankey Plot

In [ ]:
dropped_deaths_df2 = dropped_deaths_df.rename(columns={
    'in_dttm': 'earliest_location_start',
    'out_dttm': 'latest_location_end'
})

dropped_df_icu_discharge2 = dropped_df_icu_discharge.rename(columns={
    'in_dttm': 'earliest_location_start',
    'out_dttm': 'latest_location_end'
})

print(dropped_deaths_df2["hospitalization_id"].nunique())
print(dropped_df_icu_discharge2["hospitalization_id"].nunique())

In [ ]:
icu_df7_death = pd.concat([dropped_df_icu_discharge2[["patient_id","hospitalization_id","earliest_location_start",
                                                    "latest_location_end","location_category","procedural"]], 
                           dropped_deaths_df2[["patient_id","hospitalization_id","earliest_location_start",
                                                    "latest_location_end","location_category","procedural"]]], axis=0)
icu_df7_death = pd.concat([icu_df7_death, icu_df7[["patient_id","hospitalization_id","earliest_location_start",
                                                    "latest_location_end","location_category","procedural"]]], axis=0)

icu_df7_death = pd.merge(icu_df7_death,patient[["patient_id","death_dttm"]],on="patient_id",how="left")
icu_df7_death = pd.merge(icu_df7_death,hosp[["hospitalization_id","discharge_category"]],on="hospitalization_id",how="left")


icu_df7_death.head()
print("Number of rows in df:",len(icu_df7_death))
print("Number of unique patient_id:", icu_df7_death["patient_id"].nunique())
print("Number of unique hospitalization_id:", icu_df7_death["hospitalization_id"].nunique())

In [ ]:
import pandas as pd

def mark_deaths(df):
    # Ensure the columns are in datetime format
    df['death_dttm'] = pd.to_datetime(df['death_dttm'], errors='coerce')
    df['earliest_location_start'] = pd.to_datetime(df['earliest_location_start'], errors='coerce')
    df['latest_location_end'] = pd.to_datetime(df['latest_location_end'], errors='coerce')

    # Create the location_category_death column as a copy of location_category
    df['location_category_death'] = df['location_category']

    # Sort the dataframe by patient_id, hospitalization_id, and earliest_location_start to ensure order
    df = df.sort_values(by=['hospitalization_id', 'earliest_location_start'])

    # List to collect new rows that represent the death
    new_rows = []

    # Iterate over each group of hospitalization_id
    for hospitalization_id, group in df.groupby('hospitalization_id'):
        for idx, row in group.iterrows():
            # Check if death_dttm is between earliest_location_start and latest_location_end
            if (
                not pd.isna(row['death_dttm']) and 
                row['earliest_location_start'] <= row['death_dttm'] <= row['latest_location_end']
            ):
                # Create a new row with the same info but marked as 'Death'
                new_row = row.copy()
                new_row['location_category_death'] = 'Died'
                new_row['earliest_location_start'] = row['latest_location_end'] + pd.Timedelta(seconds=1)  # Add 1 second to ensure 'Death' comes after
                new_rows.append(new_row)

    # Add the new death rows to the original dataframe
    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)

    # Sort the dataframe again to make sure 'Death' comes after the original location
    df = df.sort_values(by=['hospitalization_id', 'earliest_location_start']).reset_index(drop=True)

    return df

# Sort and apply the death marking function
icu_df7_death = mark_deaths(icu_df7_death)
print("Number of dead patients:", icu_df7_death[icu_df7_death["location_category_death"] == "Died"]["hospitalization_id"].nunique())
print("Number of rows in df:", len(icu_df7_death))
print("Number of unique hospitalization_id:", icu_df7_death["hospitalization_id"].nunique())

In [ ]:
icu_df7_death[icu_df7_death["location_category_death"]=="Died"][["hospitalization_id",
                                                                  "earliest_location_start",
                                                                 "latest_location_end",
                                                                 "death_dttm",
                                                                 "location_category",
                                                                 "location_category_death"]].head()

## This code chunk takes longer to run: 

In [ ]:
def collapse_segments(df):
    # Ensure data is sorted by 'earliest_location_start'
    df = df.sort_values('earliest_location_start').reset_index(drop=True)

    # Create a new column to track if the location was ICU in the original location_category
    df['is_icu_in_location_category'] = (df['location_category'] == 'ICU')

    # Detect changes in location category, both in the original and in the death-adjusted version
    df['loc_cat_change'] = (df['location_category_death'] != df['location_category_death'].shift(1)).astype(int)

    # Assign a segment_id that increments when location_category_death changes
    df['segment_id'] = df['loc_cat_change'].cumsum()

    # Group by 'segment_id' and 'location_category_death' to merge consecutive segments
    grouped = df.groupby(['segment_id', 'location_category_death']).agg({
        'earliest_location_start': 'min',
        'latest_location_end': 'max',
        'is_icu_in_location_category': 'max'  # Capture whether any part of the segment was ICU
    }).reset_index()

    # Add 'hospitalization_id' to the grouped DataFrame
    grouped['hospitalization_id'] = df['hospitalization_id'].iloc[0]

    # Rearrange columns
    grouped = grouped[['hospitalization_id', 'segment_id', 'earliest_location_start', 'latest_location_end', 'location_category_death', 'is_icu_in_location_category']]
    
    # Initialize 'segment_rank' to None
    grouped['segment_rank'] = None

    # Now we can still find ICU segments based on the original location_category, but respect Death as a location in location_category_death
    icu_segments = grouped[grouped['is_icu_in_location_category'] == 1]

    if icu_segments.empty:
        print(f"No ICU segments for hospitalization_id: {df['hospitalization_id'].iloc[0]}")
        return pd.DataFrame()  # Return empty DataFrame to exclude this hospitalization

    # Get the index of the first ICU segment
    first_icu_index = icu_segments.index[0]

    # Create 'segment_rank' starting from the first ICU segment
    grouped.loc[first_icu_index:, 'segment_rank'] = range(1, len(grouped.loc[first_icu_index:]) + 1)

    # Fill in the 'segment_rank' for non-ICU segments after the first ICU
    grouped['segment_rank'] = grouped['segment_rank'].ffill()

    # Compute the maximum segment rank for this hospitalization
    max_segment_rank = grouped['segment_rank'].dropna().max()

    # Add 'max_segment_rank' column to the DataFrame
    grouped['max_segment_rank'] = max_segment_rank

    # Drop the auxiliary column
    grouped = grouped.drop(columns=['is_icu_in_location_category'])

    return grouped

# Apply the function to the DataFrame
collapsed_adt = icu_df7_death[icu_df7_death["procedural"] == False][['hospitalization_id', 'earliest_location_start', 'latest_location_end', 'location_category', 'location_category_death']] \
    .groupby('hospitalization_id', group_keys=False).apply(collapse_segments).reset_index(drop=True)

print("Number of unique hospitalization_id:", collapsed_adt["hospitalization_id"].nunique())
print("Number of rows in df:", len(collapsed_adt))

In [ ]:
def drop_intermediate_er2(df):
    # Initialize an empty list to hold hospitalization_id values to be dropped
    visits_to_drop = []

    # Function to process each group of 'patient_id' and 'hospitalization_id'
    def process_group(group):
        # Check if 'ER' appears after the first row in the group
        if any(group.iloc[1:]['location_category_death'] == 'ER'):
            # Mark the visit for dropping if 'ER' appears after the first row
            visits_to_drop.append(group.iloc[0]['hospitalization_id'])

    # Apply the function to each group of'hospitalization_id'
    df.groupby(['hospitalization_id']).apply(process_group)

    # Create a new DataFrame with the rows that need to be dropped
    dropped_df = df[df['hospitalization_id'].isin(visits_to_drop)]

    # Drop the visits from the original DataFrame
    df_cleaned = df[~df['hospitalization_id'].isin(visits_to_drop)]

    return df_cleaned, dropped_df

collapsed_adt = collapsed_adt.sort_values(by=['hospitalization_id', 'earliest_location_start', 'latest_location_end'])
collapsed_adt2, dropped_df = drop_intermediate_er2(collapsed_adt)

print("Number of unique hospitalization_id:", collapsed_adt2["hospitalization_id"].nunique())
print("Number of rows in df:", len(collapsed_adt2))

In [ ]:
sankey_df = collapsed_adt2.reset_index().loc[:,['hospitalization_id','segment_rank','location_category_death']]
sankey_df2 = sankey_df.loc[sankey_df.segment_rank.notna()] 
sankey_df3 = sankey_df2.pivot(
    index=['hospitalization_id'],
    columns='segment_rank',
    values='location_category_death'
).reset_index().fillna('Discharged')
sankey_df4 = sankey_df3.iloc[:, :8] #only take first 7 locations, otherwise gets too long

In [ ]:
import pandas as pd

def propagate_death(df):
    # List of columns that represent the different segments
    segment_cols = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]

    # Iterate over each row in the DataFrame
    for idx, row in df.iterrows():
        # Find the first occurrence of "Death" in the segment columns
        death_found = False
        for col in segment_cols:
            if row[col] == 'Died':
                death_found = True
            # If "Death" is found, mark all subsequent columns as "Death"
            if death_found:
                df.at[idx, col] = 'Died'
    
    return df

# Apply the function to propagate 'Death' to subsequent columns
sankey_df4 = propagate_death(sankey_df4)

In [ ]:
colors = {
    "ICU": 'lightcoral',
    "Ward": 'skyblue',
    'Procedural':'thistle',
    "Discharged": 'lightgrey',
    "Died": 'grey',
    "ER":'red'
}
fig, ax = plt.subplots(figsize=(14, 6), constrained_layout=True)
diag = Sankey(
    sankey_df4.iloc[:, 1:], 
    ax=ax, 
    order=["ICU", "Ward", "Procedural", "Discharged","ER","Died"],
    block_width=0.2,
    colors=colors,
    alpha=0.5
)
diag.draw()
ax.set_title("", size=16)
ax.set_xticks(
    [diag.block_width / 2 + diag.flow_width * x + diag.block_width * x for x in range(sankey_df4.shape[1] - 1)]
)
ax.set_xticklabels(sankey_df4.columns[1:].astype(int))
ax.set_xlabel("Location number", size=14)
ax.get_xaxis().set_visible(True)
ax.tick_params(axis="x", pad=5, labelsize=16)

# Export the figure as a high-resolution image
fig.savefig(f"{output_path}sankey_diagram_{datetime.today().date()}_{site_name}.png", dpi=300, bbox_inches='tight')

# Generate Table One

In [ ]:
unplanned_icu_final = icu_final[icu_final["procedural"]==False]
#admission location
admission_location = icu_df7[["hospitalization_id", "location_category"]].groupby('hospitalization_id', as_index=False).first()
unplanned_icu_final = pd.merge(unplanned_icu_final,admission_location,on="hospitalization_id",how="left")

#length of ICU stay 
icu_sorted = icu_df7.sort_values(by=['hospitalization_id', 'earliest_location_start'])
icu_first_icu = icu_sorted[icu_sorted['location_category'] == 'ICU'].groupby('hospitalization_id', as_index=False).first()
length_ICU = icu_first_icu[['hospitalization_id', 'location_hours']]
unplanned_icu_final = pd.merge(unplanned_icu_final,length_ICU,on="hospitalization_id",how="left")
unplanned_icu_final['location_days'] = unplanned_icu_final['location_hours'] / 24

#in-hospital mortality
dead_patients = collapsed_adt2[collapsed_adt2["location_category_death"]=="Died"][["hospitalization_id","location_category_death"]]
unplanned_icu_final = pd.merge(unplanned_icu_final,dead_patients,on="hospitalization_id",how="left")
unplanned_icu_final['location_category_death'] = unplanned_icu_final['location_category_death'].fillna('Alive')

#age at admission
hosp = hosp.drop_duplicates(subset=["hospitalization_id"])
age = hosp[["hospitalization_id","age_at_admission"]]                           
unplanned_icu_final = pd.merge(unplanned_icu_final,age,on="hospitalization_id",how="left")

#demographics 
demog = patient[["patient_id","race_category","ethnicity_category","sex_category"]]
unplanned_icu_final = pd.merge(unplanned_icu_final,demog,on="patient_id",how="left")
unplanned_icu_final = unplanned_icu_final.drop_duplicates(subset=["hospitalization_id"])

#readmitted
unplanned_icu_final['readmission'] = unplanned_icu_final['ICU_readmission_hour'].apply(
    lambda x: 'Readmitted' if pd.notna(x) and x > 0 else 'Not Readmitted'
)

print(unplanned_icu_final["hospitalization_id"].nunique())
unplanned_icu_final.head()

In [ ]:
table_one_dat = unplanned_icu_final[["age_at_admission",
                                     "sex_category",
                                     "race_category",
                                    "ethnicity_category",
                                    "location_category",
                                     "ICU_readmission_hour",
                                    "location_days",
                                    "location_category_death",
                                    "readmission"]]

# Rename columns
table_one_dat = table_one_dat.rename(columns={'age_at_admission': 'Age', 
                                              'sex_category': 'Sex', 
                                              'race_category': 'Race',
                                             'ethnicity_category': 'Ethnicity',
                                             'location_category': 'Initial hospital location',
                                              'ICU_readmission_hour':'ICU readmission, hours',
                                              "location_days":'ICU length of stay, days',
                                              "location_category_death":'In-hospital Mortality'
                                             })

In [ ]:
from tableone import TableOne

# Define the columns that include both continuous and categorical variables
columns = ["Age", "ICU readmission, hours", "ICU length of stay, days", "Sex", "Race", "Ethnicity", "Initial hospital location", "In-hospital Mortality"]

# Define which columns are categorical
categorical = ["Sex", "Race", "Ethnicity", "Initial hospital location", "In-hospital Mortality"]

# Define which continuous variables are not normally distributed
nonnormal = ["Age","ICU readmission, hours","ICU length of stay, days"]

#groupby
groupby = 'readmission'

# Generate the TableOne object
table1 = TableOne(data=table_one_dat, 
                  columns=columns, 
                  categorical=categorical, 
                  nonnormal=nonnormal, 
                  groupby=groupby)

# Display the table

table1.to_csv(f"{output_path}table1_{datetime.today().date()}_{site_name}.csv")
table1